In [1]:
#conda enviornment
#!conda activate nlp_classification

In [2]:
# Libraries
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup
import random
from datetime import date, timedelta
import time
import string

In [3]:
#Create two empty lists for the company name and company ticker symbol
company_name =[]
company_ticker = []

In [4]:
#function to scrape the NYSE tickers and company names from the site specified below
def scrape_nyse_stock_symbols(Letter):
    Letter =  Letter.upper()
    URL =  'https://www.advfn.com/nyse/newyorkstockexchange.asp?companies='+Letter
    page = requests.get(URL)    
    soup = BeautifulSoup(page.text, "html.parser")
    odd_rows = soup.find_all('tr', attrs= {'class':'ts0'})
    even_rows = soup.find_all('tr', attrs= {'class':'ts1'})
    for i in odd_rows:
        row = i.find_all('td')
        company_name.append(row[0].text.strip())
        company_ticker.append(row[1].text.strip())
    for i in even_rows:
        row = i.find_all('td')
        company_name.append(row[0].text.strip())
        company_ticker.append(row[1].text.strip())
    return (company_name, company_ticker)

In [5]:
#function to scrape the NASDAQ tickers and company names from the site specified below
def scrape_nasdaq_stock_symbols(Letter):
    Letter =  Letter.upper()
    URL =  'https://www.advfn.com/nyse/nasdaq.asp?companies='+Letter
    page = requests.get(URL)    
    soup = BeautifulSoup(page.text, "html.parser")
    odd_rows = soup.find_all('tr', attrs= {'class':'ts0'})
    even_rows = soup.find_all('tr', attrs= {'class':'ts1'})
    for i in odd_rows:
        row = i.find_all('td')
        company_name.append(row[0].text.strip())
        company_ticker.append(row[1].text.strip())
    for i in even_rows:
        row = i.find_all('td')
        company_name.append(row[0].text.strip())
        company_ticker.append(row[1].text.strip())
    return (company_name, company_ticker)

In [6]:
# Code to get NYSE tickers from site
for char in string.ascii_uppercase:
  (temp_name,temp_ticker) = scrape_nyse_stock_symbols(char)

In [7]:
# code to also get all NASDAQ tickers from site
# for char in string.ascii_uppercase:
#   (temp_name,temp_ticker) = scrape_nasdaq_stock_symbols(char)

In [8]:
#Create a new dataFrame that contains the company name and company ticker
data = pd.DataFrame(columns = ['company_name',  'company_ticker']) 
data['company_name'] = company_name
data['company_ticker'] = company_ticker


In [9]:
import yfinance as yf

In [10]:
#Getting a random sample of 2000 companies to call from API
# Note that I chose 2000 as this is the limit of API calls per hour from YFinance
test_tickers = random.sample(company_ticker,200)

In [11]:
# Creating empty dataframe to store results
stock_df = pd.DataFrame(columns=['stock_ticker','long_business_summary','full_time_employees','sector',\
                               'total_revenue','country','financial_currency','52_week_change',\
                                'debt_to_equity','ebitda_margins','current_ratio'])



In [12]:
# Code to pull stock info from YFinance API, note that some items are not present for all companies
def get_stock_info(st_ticker):
    try:
        stock = yf.Ticker(st_ticker)
        try:
            longBusinessSummary.append(stock.info['longBusinessSummary'])
        except:
            longBusinessSummary.append("")

        try:
            fullTimeEmployees.append(stock.info['fullTimeEmployees'])
        except:
            fullTimeEmployees.append("")

        try:
            sector.append(stock.info['sector'])
        except:
             sector.append("")

        try:
            country.append(stock.info['country'])
        except:
             country.append("")
        
        try:
            totalRevenue.append(stock.info['totalRevenue'])
        except:
            totalRevenue.append("")
        
        try:
            financialCurrency.append(stock.info['financialCurrency'])
        except:
            financialCurrency.append("")
            
        try:
            WeekChange_52.append(stock.info['52WeekChange'])
        except:
            WeekChange_52.append("")
            
        try:
            debtToEquity.append(stock.info['debtToEquity'])
        except:
            debtToEquity.append("")
            
        try:
            ebitdaMargins.append(stock.info['ebitdaMargins'])
        except:
            ebitdaMargins.append("")
            
        try:
            currentRatio.append(stock.info['currentRatio'])
        except:
            currentRatio.append("")
        
        time.sleep(2)
            
    except:
        longBusinessSummary.append("")
        fullTimeEmployees.append("")
        sector.append("")
        country.append("")
        totalRevenue.append("")
        financialCurrency.append("")
        WeekChange_52.append("")
        debtToEquity.append("")
        ebitdaMargins.append("")
        currentRatio.append("")
        

In [13]:
# # Loop to get company data and stor this in dataframe
for item in test_tickers:
    temp_df = pd.DataFrame()
    longBusinessSummary = []
    fullTimeEmployees = []
    sector = []
    country = []
    totalRevenue = []
    financialCurrency = []
    WeekChange_52 = []
    debtToEquity = []
    ebitdaMargins = []
    currentRatio = []
    get_stock_info(item)
    temp_df['stock_ticker'] = str(item)
    temp_df['long_business_summary'] = longBusinessSummary
    temp_df['full_time_employees'] = fullTimeEmployees
    temp_df['sector'] = sector
    temp_df['total_revenue'] = totalRevenue 
    temp_df['country'] = country
    temp_df['financial_currency'] = financialCurrency 
    temp_df['52_week_change'] = WeekChange_52
    temp_df['debt_to_equity'] = debtToEquity
    temp_df['ebitda_margins'] = ebitdaMargins
    temp_df['current_ratio'] = currentRatio
    stock_df = pd.concat([stock_df,temp_df],axis=0)

In [14]:
stock_df['stock_ticker'] = test_tickers
print(len(stock_df[stock_df['long_business_summary']!=""]))

136


In [15]:
#stock_df.to_csv("stock_info_df.csv",index=False)

In [16]:
stock_topic_df = stock_df[stock_df['long_business_summary']!=""]

In [17]:
stock_topic_df.reset_index(drop=True,inplace=True)

In [18]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import nltk    
from nltk.stem.wordnet import WordNetLemmatizer

# Neccessary functions and lists to be used in custom functions below
tokenizer = RegexpTokenizer(r'\w+')
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

# Adding additional none industry specific words that often appear in the descriptions

other_stop = ["company","service","product","wa","segment","inc","provides","offer","united","operates","states",\
           "market","fund","headquartered","solution","founded","founded","system","management","including",\
           "business","based","invests","sell","brand","income","corporation","customer","investment","new",\
           "security","portfolio","asset","name","known","incorporated","subsidiary","addition",\
           "ended","use","york","customer"]

pos_lst = ['NNP','NN','NNS','NNPS','VB','VBD','VBG','VBN','VBP','VBZ']

# Function to tokenize text as well as only keep nouns and verbs
def tokenize(text):
    tokenized = tokenizer.tokenize(text)
    nouns = []
    tokenized = tokenizer.tokenize(text)
    tagged = nltk.pos_tag(tokenized)
    for (word, tag) in tagged:
        if tag in pos_lst and len(word)>1:
            nouns.append(word)
        else:
            pass
    return nouns

def get_lemma(text_tokens):
    lemma_lst = []
    for word in text_tokens:
        lemma = WordNetLemmatizer().lemmatize(word)
        lemma_lst.append(lemma.lower())
    final_lemma_lst = [x for x in lemma_lst if x not in en_stop and x not in other_stop]
    return final_lemma_lst


[nltk_data] Downloading package stopwords to /Users/Aqqes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [19]:
stock_topic_df['tokenized'] = stock_topic_df['long_business_summary'].apply(tokenize)
stock_topic_df['lemmatized'] = stock_topic_df['tokenized'].apply(get_lemma)

/opt/anaconda3/envs/NLP/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/envs/NLP/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
# Finding the most common words that appear in each long business summary
from collections import Counter

word_lst = []
for row in stock_topic_df['lemmatized']:
    word_lst+= row
    
x = Counter(word_lst)
x.most_common()

[('equity', 43),
 ('managed', 41),
 ('gas', 39),
 ('equipment', 39),
 ('insurance', 39),
 ('industry', 37),
 ('device', 37),
 ('design', 35),
 ('manufacture', 35),
 ('consumer', 35),
 ('index', 34),
 ('power', 34),
 ('property', 34),
 ('capital', 33),
 ('oil', 33),
 ('component', 33),
 ('stock', 32),
 ('technology', 32),
 ('transportation', 32),
 ('vehicle', 31),
 ('services', 31),
 ('acquisition', 30),
 ('care', 30),
 ('home', 30),
 ('used', 28),
 ('america', 28),
 ('construction', 28),
 ('corp', 27),
 ('produce', 27),
 ('food', 27),
 ('bond', 26),
 ('government', 25),
 ('mining', 25),
 ('specialty', 25),
 ('estate', 25),
 ('health', 25),
 ('network', 24),
 ('application', 24),
 ('group', 23),
 ('performance', 23),
 ('sale', 23),
 ('europe', 23),
 ('canada', 23),
 ('california', 23),
 ('golf', 23),
 ('international', 22),
 ('launched', 22),
 ('seek', 22),
 ('formed', 22),
 ('domiciled', 22),
 ('trust', 22),
 ('invesco', 22),
 ('purchase', 21),
 ('limited', 21),
 ('north', 21),
 ('loan

In [21]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.test.utils import get_tmpfile

/opt/anaconda3/envs/NLP/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [22]:
#Doc2Vec model parama
max_epochs = 400
vec_size = 5
alpha = 0.025

model = Doc2Vec(vector_size=vec_size,
                alpha=alpha,
                min_alpha=0.00025,
                min_count=1,
                dm =1)

In [23]:
docs = []
for i in range(0,len(stock_topic_df)):
    docs.append(TaggedDocument(words=stock_topic_df.tokenized[i], tags=[i]))

In [24]:
model.build_vocab(docs)
model.train(docs, total_examples=model.corpus_count, epochs=max_epochs)

In [25]:
output_array = model.infer_vector(doc_words=stock_topic_df.lemmatized[0],alpha=0.1, min_alpha=0.0001,steps=5)

In [26]:
def get_5_doc_vectors(tokenized_text):
    output_array = model.infer_vector(doc_words=tokenized_text,alpha=alpha, min_alpha=0.00025,steps=5)
    vec1 = output_array[0]
    vec2 = output_array[1]
    vec3 = output_array[2]
    vec4 = output_array[3]
    vec5 = output_array[4]
    return vec1,vec2,vec3,vec4,vec5
    

In [27]:
stock_topic_df['vec1'], stock_topic_df['vec2'],stock_topic_df['vec3'],stock_topic_df['vec4'],stock_topic_df['vec5']\
= zip(*stock_topic_df['lemmatized'].apply(get_5_doc_vectors))

/opt/anaconda3/envs/NLP/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
stock_topic_df.head(10)

,stock_ticker,long_business_summary,full_time_employees,sector,total_revenue,country,financial_currency,52_week_change,debt_to_equity,ebitda_margins,current_ratio,tokenized,lemmatized,vec1,vec2,vec3,vec4,vec5
0,RCL,Royal Caribbean Group operates as a cruise com...,84900,Consumer Cyclical,93378000,United States,USD,0.364917,276.117,0,1.07,"[Caribbean, Group, operates, cruise, company, ...","[caribbean, group, cruise, cruise, royal, cari...",1.039310,0.682398,3.499001,1.104250,2.063452
1,TWN,"The Taiwan Fund, Inc. is a closed-ended equity...",,Financial Services,3919780,Hong Kong,USD,0.584649,NaN,0,3.045,"[Taiwan, Fund, Inc, is, ended, equity, fund, l...","[taiwan, equity, launched, managed, jf, intern...",-1.250581,-5.019403,0.314977,-6.698628,0.085419
2,GLOP,"GasLog Partners LP owns, operates, and acquire...",,Energy,315300992,Greece,USD,0.126027,125.137,0.68247,0.773,"[GasLog, Partners, LP, owns, operates, acquire...","[gaslog, partners, lp, owns, acquires, liquefi...",3.318201,2.670848,0.716679,-0.908848,3.308798
3,LFG,Rice Acquisition Corp. does not have significa...,,Utilities,None,United States,USD,0.942495,None,0,4.972,"[Rice, Acquisition, Corp, does, have, operatio...","[rice, acquisition, corp, doe, operation, focu...",1.172377,0.193258,-0.523927,-3.202907,-1.554393
4,BTCM,BIT Mining Limited operates as a cryptocurrenc...,138,Technology,2908581120,Hong Kong,CNY,2.476191,14.959,-0.02542,1.157,"[BIT, Mining, Limited, operates, cryptocurrenc...","[bit, mining, limited, cryptocurrency, mining,...",10.790651,0.600182,-3.694256,-4.438946,5.234321
5,SFL,"SFL Corporation Ltd. engages in the ownership,...",14,Industrials,452184992,Bermuda,USD,0.043709,249.118,0.65464,1.079,"[Corporation, Ltd, engages, ownership, operati...","[ltd, engages, ownership, operation, charterin...",4.521241,6.685973,1.978058,-6.390811,7.464607
6,BIF,"Boulder Growth & Income Fund, Inc. is a closed...",,Financial Services,22508436,United States,USD,0.353831,14.136,0,0.479,"[Boulder, Growth, Income, Fund, Inc, is, ended...","[boulder, growth, balanced, launched, managed,...",0.040376,-7.950470,2.339500,-8.087067,3.431307
7,STE,STERIS plc provides infection prevention and o...,13000,Healthcare,3407009024,Ireland,USD,0.294636,58.29,0.26417,1.754,"[STERIS, plc, provides, infection, prevention,...","[steris, plc, infection, prevention, worldwide...",-1.581533,4.254619,-0.129259,-8.857157,-8.087415
8,TPGY,TPG Pace Beneficial Finance Corp. focuses on e...,,Financial Services,None,United States,USD,0.092213,None,0,0.005,"[TPG, Pace, Beneficial, Finance, Corp, focuses...","[tpg, pace, beneficial, finance, corp, focus, ...",-1.275671,-1.536415,2.048870,-2.232834,-1.640962
9,BITE,Bite Acquisition Corp. focuses on effecting a ...,,Financial Services,None,United States,USD,-0.010204,None,0,0.178,"[Bite, Acquisition, Corp, focuses, effecting, ...","[bite, acquisition, corp, focus, effecting, me...",-0.685834,-1.474573,0.738550,-4.976618,-0.531622


In [29]:
stock_topic_df.to_csv("public_company_topic_modeling_data.csv",index=False)

In [30]:
# Code to save and load model
fname = get_tmpfile("1400_sample_doc2vec_model")
model.save(fname)
# model = Doc2Vec.load(fname)